<a href="https://colab.research.google.com/github/nazmul-kaonine/Data-Science-Projects/blob/master/Comparison%20between%20different%20movie%20recommendation%20algorithms%20and%20creation%20of%20a%20new%20recommendation%20model%20using%20machine%20learning/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Comparison between different movie Recommender Systems and creation of a new recommendation model.

###Coded by: Nazmul Kaonine


###Research members:
###Rebeka Gadzo


###Gurliv Pal Singh


###Smita Naik




###This code contains:

Collaborative Filtering:  
*   User-User Similarity (6 similarities)
*   Item Item-Item Similarity (6 similarities)
##Similarities:
####   *Jaccard
####   *Cosine
####   *Euclidean
####   *Correlation
####   *l1 regularization
####   *l2 regularization

##Analysis on the movielens dataset provided by GroupLens: 
####  *Similarity Matrices
####   *Recommended movies from every method

##MovienChill!- Our own Recommender System
               Algorithm used- RandomForest
               Feature Tuning &
               Numerical Conversion

##Evaluation using:
#####*RMSD
#####*Recall
#####*Accuracy
               



         


#####------------------Run all---------------------

##Loading Required Libraries

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation, jaccard, euclidean

from math import sqrt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


##Loading the Data

In [ ]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip



--2020-06-14 11:31:21--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.4’

ml-100k.zip.4       100%[===================>]   4.70M  3.46MB/s    in 1.4s    

2020-06-14 11:31:23 (3.46 MB/s) - ‘ml-100k.zip.4’ saved [4924029/4924029]

Archive:  ml-100k.zip
replace ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!cd /content/ml-100k #checking path

##Getting Ratings table

In [ ]:
ratings= pd.read_csv( "/content/ml-100k/u.data", delimiter = "\t", header = None )

In [ ]:
ratings.columns=["userid", "movieid", "rating", "timestamp"]

In [ ]:
ratings.head(5)

,userid,movieid,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
movies= pd.read_csv( "/content/ml-100k/u.item", delimiter = '\|', header=None )


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [ ]:
movies.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [ ]:
movies = movies.iloc[:,:2]
movies.columns = ['movieid', 'title']
movies.head(5)
movies.loc[movies['movieid'] == 1] #movie name with movieid=1

,movieid,title
0,1,Toy Story (1995)


##Creating the Mixed Table

In [ ]:
ratings=pd.merge(ratings,movies,on="movieid",how="inner")
ratings.head()

,userid,movieid,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


###We have the first column as the user ids and the first row as the movie ids. So, we have basically achieved a table that shows each user's rating on each movie.

In [ ]:
movies.loc[movies['movieid'] == 5] #checking the 5th movie name

,movieid,title
4,5,Copycat (1995)


#USER-BASED COLLABORATIVE FILTERING

In [ ]:
#USER-ITEM MATRIX
ratings_user_matrix=ratings.pivot_table(index=["userid"],columns=["movieid"],values="rating").reset_index(drop=True)
ratings_user_matrix.fillna(0,inplace=True)
ratings_user_matrix.head()

movieid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ratings_user_matrix.fillna(0, inplace = True) #Replacing missing values with 0

In [ ]:
ratings_user_matrix.shape #943 users and 1682 movies

(943, 1682)

###User-User Similarity Matrix

In [ ]:
user_similarity = 1 - pairwise_distances(ratings_user_matrix.to_numpy(), metric="jaccard" )
np.fill_diagonal(user_similarity, 0 ) 
user_user_df= pd.DataFrame(user_similarity)
user_user_df

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942
0,0.000000,0.056962,0.025157,0.024221,0.217984,0.248062,0.273585,0.114478,0.017301,0.203166,0.185864,0.117647,0.237057,0.149068,0.096210,0.215339,0.063830,0.279720,0.031802,0.095890,0.102689,0.186944,0.258929,0.122112,0.143791,0.114706,0.031250,0.135922,0.040816,0.067797,0.037037,0.075601,0.013699,0.010381,0.024138,0.010381,0.111486,0.135838,0.013793,0.033670,...,0.045902,0.040000,0.046823,0.129380,0.119741,0.027586,0.106667,0.138462,0.076159,0.231707,0.017241,0.010169,0.357143,0.044218,0.122754,0.204433,0.017065,0.143713,0.216463,0.067901,0.123810,0.034014,0.020979,0.149560,0.066667,0.099315,0.127946,0.067308,0.209906,0.291785,0.208672,0.050676,0.131148,0.064846,0.104651,0.049020,0.159021,0.035211,0.083333,0.232493
1,0.056962,0.000000,0.084112,0.088608,0.021552,0.132780,0.040268,0.052174,0.076923,0.069565,0.056522,0.056075,0.067278,0.118881,0.248120,0.063158,0.111111,0.059375,0.064935,0.037736,0.085586,0.027027,0.065000,0.101695,0.068702,0.224638,0.087500,0.052239,0.090909,0.129032,0.042553,0.119565,0.088608,0.123288,0.035714,0.064935,0.034783,0.033898,0.166667,0.141176,...,0.101010,0.108696,0.144444,0.142077,0.046154,0.086420,0.196078,0.012658,0.008772,0.089888,0.024096,0.157895,0.044077,0.154762,0.024845,0.167364,0.173333,0.102564,0.061798,0.133333,0.133858,0.021739,0.205882,0.064327,0.044444,0.057143,0.146789,0.308511,0.016779,0.033613,0.067873,0.160920,0.192982,0.243902,0.164384,0.110000,0.111842,0.090909,0.093023,0.040724
2,0.025157,0.084112,0.000000,0.200000,0.004386,0.039216,0.031603,0.066038,0.027027,0.034783,0.039823,0.029412,0.073095,0.027027,0.074830,0.031915,0.037975,0.012232,0.088235,0.030303,0.088785,0.016760,0.019900,0.051724,0.015385,0.087838,0.025974,0.055556,0.128205,0.054348,0.111111,0.079545,0.238095,0.104478,0.144928,0.088235,0.009091,0.029412,0.206349,0.253521,...,0.041237,0.146341,0.032609,0.086486,0.049180,0.052632,0.055556,0.006623,0.019048,0.056818,0.000000,0.194030,0.013661,0.022989,0.006410,0.124481,0.269841,0.044586,0.028409,0.103448,0.046154,0.131579,0.193548,0.023529,0.048780,0.019802,0.026316,0.105769,0.000000,0.012766,0.013333,0.021978,0.088889,0.068182,0.065789,0.019802,0.102740,0.055556,0.072581,0.009091
3,0.024221,0.088608,0.200000,0.000000,0.010152,0.026201,0.028916,0.106667,0.045455,0.019608,0.040609,0.041667,0.031250,0.033898,0.049180,0.031447,0.019608,0.010067,0.100000,0.074627,0.062827,0.034014,0.023392,0.095238,0.030303,0.056452,0.042553,0.061856,0.094340,0.063492,0.052632,0.065574,0.200000,0.100000,0.139535,0.047619,0.051948,0.028369,0.121951,0.134615,...,0.044118,0.084746,0.015625,0.049080,0.053763,0.041667,0.063291,0.016667,0.013158,0.054054,0.000000,0.111111,0.014881,0.035088,0.000000,0.061674,0.111111,0.038760,0.048611,0.042553,0.039216,0.076923,0.128205,0.043478,0.056604,0.028169,0.048193,0.062500,0.007605,0.019608,0.015385,0.016129,0.050633,0.103448,0.056000,0.013889,0.082645,0.095238,0.084211,0.021277
4,0.217984,0.021552,0.004386,0.010152,0.000000,0.122093,0.214286,0.109005,0.020725,0.108025,0.202703,0.071090,0.187408,0.100806,0.052830,0.145455,0.025253,0.170984,0.026316,0.115000,0.109718,0.267782,0.239544,0.075221,0.134529,0.072243,0.020408,0.159817,0.019512,0.063415,0.014423,0.058824,0.000000,0.005155,0.010101,0.000000,0.094340,0.218107,0.000000,0.009615,...,0.037383,0.004673,0.018868,0.076923,0.106667,0.005000,0.073059,0.176724,0.070423,0.167300,0.031250,0.000000,0.179856,0.029412,0.098814,0.091922,0.000000,0.135458,0.258333,0.068670,0.088983,0.024752,0.000000,0.184739,0.029851,0.087379,0.076923,0.026087,0.175141,0.264085,0.199313,0.028846,0.049669,0.023810,0.080153,0.032258,0.110236,0.031414,0.071730,0.207746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
user_similarity = 1 - pairwise_distances(ratings_user_matrix.to_numpy(), metric="l2" )
np.fill_diagonal(user_similarity, 0 ) 
user_user_df= pd.DataFrame(user_similarity)
user_user_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942
0,0.000000,-64.253352,-64.916614,-64.360539,-60.147363,-62.348244,-77.689262,-59.695964,-63.776539,-66.527772,-65.430415,-60.057350,-80.178815,-62.031738,-64.444633,-62.780875,-60.684682,-65.309879,-62.395583,-60.684682,-66.601775,-59.868711,-58.253692,-62.150614,-60.286214,-62.047601,-62.126856,-60.967734,-63.031242,-62.592452,-63.907627,-61.289646,-64.107603,-64.368188,-63.443774,-64.099923,-60.546730,-68.713700,-63.629715,-63.117080,...,-64.559134,-63.039051,-63.498062,-72.878278,-61.153037,-63.459289,-60.627916,-62.929649,-62.718129,-57.625933,-63.163853,-63.443774,-57.060313,-62.812225,-63.876806,-66.297845,-63.342832,-62.466527,-59.282667,-66.564784,-61.128898,-63.280635,-62.639610,-65.294796,-61.481997,-60.229078,-60.032778,-64.719099,-70.874891,-54.614746,-63.591021,-64.030762,-66.111847,-61.841069,-65.279710,-65.775744,-61.369865,-62.174362,-66.742158,-60.895073
1,-64.253352,0.000000,-34.623026,-32.970576,-49.009999,-54.812185,-83.699469,-39.816663,-32.734256,-59.646517,-52.563047,-40.206796,-83.439327,-44.705580,-31.726136,-56.375953,-30.288976,-66.609171,-31.908965,-35.864617,-43.877611,-48.769469,-50.390661,-41.918527,-42.611925,-31.664966,-30.796226,-42.266615,-33.117444,-33.842503,-36.188708,-32.376639,-31.695565,-31.202484,-32.060551,-32.496268,-38.924930,-51.345009,-29.870698,-30.096624,...,-34.721142,-31.878564,-32.376639,-55.071383,-40.194660,-33.856850,-31.372828,-47.631266,-40.182521,-46.801674,-32.689761,-29.512293,-64.749525,-31.000000,-44.705580,-49.209561,-28.461840,-42.220366,-45.882833,-40.460825,-39.174619,-33.669872,-27.106939,-48.162994,-37.574603,-37.065733,-32.331667,-28.715316,-67.963759,-46.169906,-53.626001,-31.848135,-44.166359,-28.137605,-38.153544,-36.828561,-41.355637,-32.241540,-43.710178,-52.244718
2,-64.916614,-34.623026,0.000000,-24.238859,-46.222876,-56.775427,-82.958323,-35.428011,-28.410882,-59.423505,-50.633323,-36.669616,-83.350459,-45.540305,-35.851052,-55.008928,-26.748874,-68.079664,-25.267851,-30.208973,-41.367440,-45.454279,-49.695167,-40.243181,-40.844952,-34.693137,-26.676705,-38.534795,-27.792360,-31.078030,-28.664794,-28.342802,-21.803509,-26.313001,-22.958297,-25.739484,-35.041643,-47.548944,-23.494897,-21.671568,...,-32.481338,-26.276363,-31.326460,-56.436922,-35.823905,-29.594117,-32.421550,-43.631827,-34.707142,-45.238512,-26.495454,-21.583180,-63.930732,-29.708305,-41.071368,-50.845926,-20.424285,-39.804412,-43.463468,-38.293765,-39.062451,-25.210685,-22.216374,-45.818800,-32.689761,-33.263683,-32.286634,-33.029399,-66.253253,-43.586994,-53.543561,-31.893768,-46.822589,-29.724583,-39.224371,-36.443290,-39.236799,-27.106939,-40.880783,-50.613952
3,-64.360539,-32.970576,-24.238859,0.000000,-45.701178,-56.645468,-82.282651,-33.058773,-27.284271,-59.299254,-49.803543,-36.148351,-85.011627,-44.683695,-35.152455,-54.226805,-25.888659,-67.840395,-23.959968,-29.149627,-40.617304,-44.199558,-48.789557,-37.858718,-39.595566,-34.594943,-25.400758,-36.576588,-27.495614,-30.016125,-30.320920,-27.774989,-22.173260,-25.627054,-23.145393,-26.166155,-32.852622,-47.207883,-23.919872,-23.637370,...,-31.893768,-26.658633,-31.465366,-56.706152,-35.013886,-28.949958,-30.176915,-42.897608,-34.327043,-44.574115,-25.925824,-24.436195,-63.536811,-29.232433,-40.821047,-52.525695,-24.139610,-39.373258,-41.614552,-39.385641,-38.471509,-24.961510,-22.452079,-44.022217,-32.555923,-32.630343,-30.984371,-33.394767,-65.543219,-42.577517,-52.870214,-31.542280,-47.249352,-27.160256,-38.484174,-35.972963,-38.242834,-25.814175,-39.853396,-49.685304
4,-60.147363,-49.009999,-46.222876,-45.701178,0.000000,-59.728906,-76.272246,-44.464272,-44.727453,-63.117080,-51.839379,-48.325450,-80.553663,-50.516987,-49.517324,-56.175169,-43.045431,-6

In [ ]:
user_similarity = 1 - pairwise_distances(ratings_user_matrix.to_numpy(), metric="correlation" )
np.fill_diagonal(user_similarity, 0 ) 
user_user_df= pd.DataFrame(user_similarity)
user_user_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942
0,0.000000,0.106322,-0.016424,0.021716,0.302592,0.345976,0.320017,0.276208,0.038812,0.288491,0.217507,0.273815,0.296534,0.270445,0.106351,0.335007,0.184958,0.380732,0.074695,0.198485,0.098842,0.318704,0.379766,0.231979,0.281809,0.178358,0.092360,0.246582,0.083585,0.134798,0.066344,0.166742,-0.013125,-0.011084,-0.008550,-0.013843,0.232300,0.113773,0.016628,0.034688,...,0.056125,0.072093,0.087437,0.139975,0.218879,0.075943,0.211707,0.218533,0.152117,0.377348,0.017865,0.002821,0.487949,0.097007,0.159046,0.238376,0.014200,0.206190,0.317505,0.088004,0.244603,0.031617,0.047844,0.179413,0.199129,0.238361,0.231222,0.083730,0.249563,0.426137,0.285923,0.068878,0.189272,0.146979,0.119444,0.060175,0.247483,0.115504,0.108984,0.323231
1,0.106322,0.000000,0.082680,0.160323,0.020218,0.197844,0.022886,0.072772,0.143716,0.106861,0.075766,0.092465,0.147474,0.191993,0.432019,0.077818,0.206313,0.117309,0.083600,0.032104,0.159973,0.018688,0.098722,0.147140,0.113763,0.420319,0.167306,0.066191,0.153623,0.207484,0.057122,0.193653,0.148923,0.197814,0.055391,0.089865,0.048516,0.013603,0.253834,0.232400,...,0.186997,0.202837,0.257415,0.228831,0.066123,0.134579,0.322636,-0.018693,-0.009086,0.127481,0.019657,0.244619,0.043629,0.266140,0.006593,0.339394,0.314020,0.128491,0.077631,0.201167,0.213287,0.024560,0.379684,0.070114,0.067676,0.086790,0.263913,0.495166,-0.022052,0.049557,0.106416,0.288550,0.325233,0.408466,0.289112,0.204094,0.190848,0.144177,0.138292,0.054235
2,-0.016424,0.082680,0.000000,0.331674,-0.028322,0.017673,-0.013384,0.055385,0.043505,0.012080,0.025442,0.040268,0.129876,0.004462,0.096585,0.013078,0.011647,-0.034706,0.088497,0.025274,0.124820,-0.007348,-0.017713,0.040256,0.002089,0.129180,0.001982,0.034740,0.133598,0.085206,0.170915,0.103694,0.375219,0.125681,0.232909,0.117553,-0.017218,0.022689,0.271927,0.371283,...,0.038628,0.200695,0.038838,0.093545,0.051080,0.052404,0.019433,-0.030587,0.006215,0.046744,-0.018186,0.345865,-0.025589,0.022542,-0.033843,0.224201,0.423346,0.057450,0.012516,0.134918,0.035108,0.166592,0.274150,0.011324,0.048688,0.009863,-0.006934,0.150569,-0.065909,-0.014137,-0.020605,0.018746,0.124174,0.046247,0.088974,-0.001145,0.127539,0.084532,0.102025,-0.023601
3,0.021716,0.160323,0.331674,0.000000,-0.002426,0.030206,0.041056,0.171116,0.089372,0.024018,0.071262,0.056481,0.066668,0.044836,0.126110,0.054693,0.040303,-0.011837,0.148089,0.069366,0.163750,0.054926,0.029479,0.156495,0.061020,0.127635,0.061360,0.130522,0.129412,0.128847,0.054866,0.116582,0.329946,0.141741,0.185558,0.055604,0.092868,0.037886,0.218010,0.226335,...,0.055522,0.156490,0.010156,0.085330,0.084895,0.068612,0.133391,0.002384,0.012381,0.079686,-0.012844,0.124093,0.006625,0.028358,-0.028503,0.152497,0.159370,0.074810,0.106218,0.072190,0.058690,0.155011,0.222635,0.101712,0.038013,0.029559,0.053381,0.120145,-0.021349,0.038206,0.016641,0.019738,0.105135,0.179598,0.122649,0.010890,0.174904,0.140811,0.149575,0.025367
4,0.302592,0.020218,-0.028322,-0.002426,0.000000,0.153409,0.279677,0.209280,0.025496,0.117166,0.252772,0.085580,0.229683,0.189778,0.032820,0.224095,0.052050,0.198013,0.090244,0.205944,0.099764,0.412184,0.354731,0.085329,0.286415,0.087925,0.048942,0.279251,0.012533,0.126665,-0.000172,0.128379,-0.036292,-0.029068,-0.026161,-0.031751,0.203441,0.197375,-0.034123,-0.036586,...,0.001306,-0.022650,0.018626,0.065885,0.182872,-0.009780,0.148305,0.303923,0.132861,0.278610,0.007465,-0.036568,0.253080,0.036331,0.107855,0.094282,-0.037063,0.202748,0.375348,0.105465,0.158590,0.026047,-0.032250,0.232972,0.047889,0.196578,0.096518,0.015334,0.244056,0.313342,0.272866,0.039333,0.013999,0.039519,0.085548,0.024292,0.182466,0.112571,0.096603,0.248991
...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
user_similarity = 1 - pairwise_distances(ratings_user_matrix.to_numpy(), metric="euclidean" )
np.fill_diagonal(user_similarity, 0 ) 
user_user_df= pd.DataFrame(user_similarity)
user_user_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942
0,0.000000,-64.253352,-64.916614,-64.360539,-60.147363,-62.348244,-77.689262,-59.695964,-63.776539,-66.527772,-65.430415,-60.057350,-80.178815,-62.031738,-64.444633,-62.780875,-60.684682,-65.309879,-62.395583,-60.684682,-66.601775,-59.868711,-58.253692,-62.150614,-60.286214,-62.047601,-62.126856,-60.967734,-63.031242,-62.592452,-63.907627,-61.289646,-64.107603,-64.368188,-63.443774,-64.099923,-60.546730,-68.713700,-63.629715,-63.117080,...,-64.559134,-63.039051,-63.498062,-72.878278,-61.153037,-63.459289,-60.627916,-62.929649,-62.718129,-57.625933,-63.163853,-63.443774,-57.060313,-62.812225,-63.876806,-66.297845,-63.342832,-62.466527,-59.282667,-66.564784,-61.128898,-63.280635,-62.639610,-65.294796,-61.481997,-60.229078,-60.032778,-64.719099,-70.874891,-54.614746,-63.591021,-64.030762,-66.111847,-61.841069,-65.279710,-65.775744,-61.369865,-62.174362,-66.742158,-60.895073
1,-64.253352,0.000000,-34.623026,-32.970576,-49.009999,-54.812185,-83.699469,-39.816663,-32.734256,-59.646517,-52.563047,-40.206796,-83.439327,-44.705580,-31.726136,-56.375953,-30.288976,-66.609171,-31.908965,-35.864617,-43.877611,-48.769469,-50.390661,-41.918527,-42.611925,-31.664966,-30.796226,-42.266615,-33.117444,-33.842503,-36.188708,-32.376639,-31.695565,-31.202484,-32.060551,-32.496268,-38.924930,-51.345009,-29.870698,-30.096624,...,-34.721142,-31.878564,-32.376639,-55.071383,-40.194660,-33.856850,-31.372828,-47.631266,-40.182521,-46.801674,-32.689761,-29.512293,-64.749525,-31.000000,-44.705580,-49.209561,-28.461840,-42.220366,-45.882833,-40.460825,-39.174619,-33.669872,-27.106939,-48.162994,-37.574603,-37.065733,-32.331667,-28.715316,-67.963759,-46.169906,-53.626001,-31.848135,-44.166359,-28.137605,-38.153544,-36.828561,-41.355637,-32.241540,-43.710178,-52.244718
2,-64.916614,-34.623026,0.000000,-24.238859,-46.222876,-56.775427,-82.958323,-35.428011,-28.410882,-59.423505,-50.633323,-36.669616,-83.350459,-45.540305,-35.851052,-55.008928,-26.748874,-68.079664,-25.267851,-30.208973,-41.367440,-45.454279,-49.695167,-40.243181,-40.844952,-34.693137,-26.676705,-38.534795,-27.792360,-31.078030,-28.664794,-28.342802,-21.803509,-26.313001,-22.958297,-25.739484,-35.041643,-47.548944,-23.494897,-21.671568,...,-32.481338,-26.276363,-31.326460,-56.436922,-35.823905,-29.594117,-32.421550,-43.631827,-34.707142,-45.238512,-26.495454,-21.583180,-63.930732,-29.708305,-41.071368,-50.845926,-20.424285,-39.804412,-43.463468,-38.293765,-39.062451,-25.210685,-22.216374,-45.818800,-32.689761,-33.263683,-32.286634,-33.029399,-66.253253,-43.586994,-53.543561,-31.893768,-46.822589,-29.724583,-39.224371,-36.443290,-39.236799,-27.106939,-40.880783,-50.613952
3,-64.360539,-32.970576,-24.238859,0.000000,-45.701178,-56.645468,-82.282651,-33.058773,-27.284271,-59.299254,-49.803543,-36.148351,-85.011627,-44.683695,-35.152455,-54.226805,-25.888659,-67.840395,-23.959968,-29.149627,-40.617304,-44.199558,-48.789557,-37.858718,-39.595566,-34.594943,-25.400758,-36.576588,-27.495614,-30.016125,-30.320920,-27.774989,-22.173260,-25.627054,-23.145393,-26.166155,-32.852622,-47.207883,-23.919872,-23.637370,...,-31.893768,-26.658633,-31.465366,-56.706152,-35.013886,-28.949958,-30.176915,-42.897608,-34.327043,-44.574115,-25.925824,-24.436195,-63.536811,-29.232433,-40.821047,-52.525695,-24.139610,-39.373258,-41.614552,-39.385641,-38.471509,-24.961510,-22.452079,-44.022217,-32.555923,-32.630343,-30.984371,-33.394767,-65.543219,-42.577517,-52.870214,-31.542280,-47.249352,-27.160256,-38.484174,-35.972963,-38.242834,-25.814175,-39.853396,-49.685304
4,-60.147363,-49.009999,-46.222876,-45.701178,0.000000,-59.728906,-76.272246,-44.464272,-44.727453,-63.117080,-51.839379,-48.325450,-80.553663,-50.516987,-49.517324,-56.175169,-43.045431,-6

In [ ]:
user_similarity = 1 - pairwise_distances(ratings_user_matrix.to_numpy(), metric="l1" )
np.fill_diagonal(user_similarity, 0 ) 
user_user_df= pd.DataFrame(user_similarity)
user_user_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942
0,0.0,-1077.0,-1098.0,-1045.0,-1012.0,-1092.0,-1591.0,-943.0,-1033.0,-1169.0,-1160.0,-945.0,-1873.0,-1020.0,-1110.0,-1027.0,-958.0,-1204.0,-1000.0,-972.0,-1229.0,-968.0,-946.0,-999.0,-965.0,-1062.0,-1000.0,-979.0,-1023.0,-1011.0,-1046.0,-977.0,-1052.0,-1046.0,-1036.0,-1045.0,-967.0,-1193.0,-1036.0,-1036.0,...,-1075.0,-1034.0,-1041.0,-1309.0,-1000.0,-1028.0,-969.0,-1044.0,-1025.0,-932.0,-1028.0,-1042.0,-1008.0,-1021.0,-1096.0,-1222.0,-1035.0,-1051.0,-961.0,-1126.0,-1001.0,-1031.0,-1009.0,-1118.0,-971.0,-954.0,-966.0,-1084.0,-1305.0,-886.0,-1099.0,-1050.0,-1167.0,-998.0,-1126.0,-1094.0,-1019.0,-992.0,-1136.0,-1024.0
1,-1077.0,0.0,-340.0,-285.0,-692.0,-804.0,-1705.0,-411.0,-281.0,-879.0,-766.0,-403.0,-1933.0,-506.0,-320.0,-751.0,-258.0,-1152.0,-274.0,-358.0,-589.0,-624.0,-688.0,-433.0,-475.0,-338.0,-264.0,-477.0,-299.0,-311.0,-342.0,-297.0,-272.0,-254.0,-288.0,-275.0,-407.0,-647.0,-244.0,-266.0,...,-333.0,-292.0,-291.0,-711.0,-450.0,-292.0,-287.0,-590.0,-423.0,-596.0,-292.0,-244.0,-1188.0,-265.0,-548.0,-696.0,-233.0,-499.0,-587.0,-422.0,-421.0,-317.0,-209.0,-606.0,-347.0,-370.0,-312.0,-244.0,-1145.0,-662.0,-767.0,-280.0,-531.0,-228.0,-424.0,-352.0,-487.0,-274.0,-480.0,-734.0
2,-1098.0,-340.0,0.0,-182.0,-645.0,-867.0,-1688.0,-344.0,-232.0,-884.0,-739.0,-354.0,-1920.0,-533.0,-399.0,-726.0,-227.0,-1203.0,-199.0,-285.0,-548.0,-565.0,-681.0,-416.0,-454.0,-401.0,-225.0,-416.0,-232.0,-284.0,-241.0,-252.0,-157.0,-203.0,-179.0,-198.0,-354.0,-574.0,-173.0,-173.0,...,-308.0,-225.0,-288.0,-748.0,-383.0,-245.0,-322.0,-519.0,-340.0,-571.0,-221.0,-163.0,-1185.0,-264.0,-493.0,-747.0,-150.0,-472.0,-552.0,-397.0,-428.0,-210.0,-164.0,-573.0,-280.0,-319.0,-329.0,-321.0,-1106.0,-621.0,-776.0,-291.0,-598.0,-265.0,-455.0,-351.0,-454.0,-219.0,-435.0,-709.0
3,-1045.0,-285.0,-182.0,0.0,-592.0,-836.0,-1629.0,-271.0,-179.0,-845.0,-684.0,-305.0,-1973.0,-476.0,-358.0,-669.0,-180.0,-1156.0,-148.0,-232.0,-507.0,-500.0,-626.0,-339.0,-391.0,-372.0,-172.0,-349.0,-195.0,-233.0,-226.0,-211.0,-132.0,-158.0,-146.0,-167.0,-279.0,-525.0,-150.0,-162.0,...,-259.0,-200.0,-253.0,-717.0,-332.0,-200.0,-255.0,-462.0,-297.0,-524.0,-174.0,-162.0,-1134.0,-217.0,-448.0,-760.0,-159.0,-429.0,-481.0,-378.0,-385.0,-175.0,-137.0,-498.0,-237.0,-272.0,-272.0,-294.0,-1045.0,-560.0,-723.0,-248.0,-581.0,-200.0,-406.0,-304.0,-405.0,-164.0,-378.0,-648.0
4,-1012.0,-692.0,-645.0,-592.0,0.0,-1011.0,-1514.0,-584.0,-576.0,-1056.0,-801.0,-650.0,-1846.0,-717.0,-735.0,-846.0,-557.0,-1195.0,-537.0,-531.0,-828.0,-559.0,-667.0,-706.0,-594.0,-713.0,-557.0,-580.0,-604.0,-584.0,-623.0,-566.0,-591.0,-581.0,-573.0,-578.0,-574.0,-734.0,-585.0,-599.0,...,-652.0,-625.0,-628.0,-1026.0,-619.0,-605.0,-594.0,-609.0,-608.0,-699.0,-557.0,-583.0,-1129.0,-592.0,-725.0,-1057.0,-586.0,-672.0,-590.0,-701.0,-674.0,-574.0,-568.0,-701.0,-608.0,-565.0,-611.0,-691.0,-1058.0,-653.0,-806.0,-617.0,-946.0,-603.0,-747.0,-671.0,-702.0,-543.0,-733.0,-769.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,-1094.0,-352.0,-351.0,-304.0,-671.0,-897.0,-1688.0,-396.0,-292.0,-930.0,-753.0,-394.0,-2026.0,-549.0,-339.0,-746.0,-267.0,-1209.0,-271.0,-293.0,-588.0,-587.0,-699.0,-442.0,-470.0,-387.0,-247.0,-484.0,-324.0,-340.0,-349.0,-274.0,-289.0,-289.0,-265.0,-284.0,-366.0,-572.0,-283.0,-303.0,...,-328.0,-283.0,-298.0,-688.0,-455.0,-305.0,-310.0,-581.0,-402.0,-615.0,-279.0,-285.0,-1193.0,-258.0,-545.0,-803.0,-284.0,-466.0,-566.0,-401.0,-442.0,-308.0,-260.0,-533.0,-334.0,-379.0,-323.0,-331.0,-1108.

In [ ]:
user_similarity = 1 - pairwise_distances(ratings_user_matrix.to_numpy(), metric="cosine" )
np.fill_diagonal(user_similarity, 0 ) 
user_user_df= pd.DataFrame(user_similarity)
user_user_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942
0,0.000000,0.166931,0.047460,0.064358,0.378475,0.430239,0.440367,0.319072,0.078138,0.376544,0.311124,0.314678,0.439743,0.332107,0.181473,0.403987,0.215819,0.473875,0.109084,0.241498,0.198413,0.381042,0.446288,0.285399,0.335553,0.252066,0.129579,0.302737,0.129084,0.181933,0.114717,0.209719,0.032820,0.029046,0.036318,0.026590,0.278971,0.195617,0.057517,0.083809,...,0.112985,0.122914,0.138209,0.234845,0.276285,0.116184,0.260551,0.285488,0.204004,0.437751,0.057587,0.047805,0.569066,0.140963,0.232308,0.338279,0.058859,0.276885,0.383251,0.158199,0.302857,0.079341,0.083955,0.257381,0.234315,0.280072,0.279170,0.146437,0.354255,0.490353,0.369527,0.119482,0.274876,0.189705,0.197326,0.118095,0.314072,0.148617,0.179508,0.398175
1,0.166931,0.000000,0.110591,0.178121,0.072979,0.245843,0.107328,0.103344,0.161048,0.159862,0.129257,0.121268,0.223661,0.226758,0.455361,0.125330,0.223667,0.179218,0.101374,0.060499,0.204445,0.064443,0.146500,0.178323,0.148638,0.445594,0.185077,0.102455,0.175115,0.230078,0.081875,0.216032,0.167204,0.212489,0.075602,0.107012,0.079821,0.058597,0.268489,0.251593,...,0.211427,0.224852,0.278382,0.269172,0.101681,0.153909,0.345035,0.025506,0.023080,0.170131,0.039270,0.260726,0.116597,0.284455,0.049835,0.376975,0.328677,0.167278,0.122291,0.231709,0.244595,0.048471,0.390525,0.113863,0.091148,0.114631,0.288602,0.512473,0.047879,0.102243,0.156986,0.307942,0.358789,0.424046,0.319889,0.228583,0.226790,0.161485,0.172268,0.105798
2,0.047460,0.110591,0.000000,0.344151,0.021245,0.072415,0.066137,0.083060,0.061040,0.065151,0.075912,0.067345,0.198196,0.042780,0.129912,0.058530,0.031320,0.033788,0.104290,0.050696,0.165823,0.034462,0.030955,0.071430,0.037031,0.163477,0.021322,0.068035,0.153260,0.108480,0.190257,0.125907,0.386958,0.140309,0.247316,0.132368,0.012527,0.062250,0.284856,0.385311,...,0.064352,0.220405,0.063105,0.136337,0.083157,0.071430,0.048256,0.009207,0.034366,0.088333,0.000000,0.358316,0.046124,0.044585,0.006246,0.264406,0.434379,0.094758,0.055179,0.164234,0.069298,0.184687,0.286220,0.052784,0.070027,0.036707,0.023085,0.176416,0.000000,0.036302,0.031875,0.042753,0.163829,0.069038,0.124245,0.026271,0.161890,0.101243,0.133416,0.026556
3,0.064358,0.178121,0.344151,0.000000,0.031804,0.068044,0.091230,0.188060,0.101284,0.060859,0.104182,0.075324,0.120306,0.070670,0.148397,0.084951,0.053895,0.035582,0.158678,0.086565,0.189713,0.082280,0.061922,0.175470,0.084112,0.151322,0.074295,0.151293,0.143465,0.144516,0.070563,0.132090,0.339211,0.152106,0.196643,0.067042,0.111555,0.065307,0.228181,0.238689,...,0.073434,0.171210,0.027923,0.115243,0.106580,0.081942,0.151247,0.029705,0.032256,0.107485,0.000000,0.136295,0.055061,0.043939,0.000000,0.182571,0.171236,0.100374,0.132542,0.094344,0.082147,0.168100,0.232185,0.127523,0.053364,0.048234,0.073335,0.138921,0.023274,0.071446,0.052107,0.036784,0.133115,0.193471,0.146058,0.030138,0.196858,0.152041,0.170086,0.058752
4,0.378475,0.072979,0.021245,0.031804,0.000000,0.237286,0.373600,0.248930,0.056847,0.201427,0.321710,0.130664,0.345217,0.244741,0.096349,0.287556,0.084729,0.289495,0.116843,0.240738,0.176494,0.455057,0.409268,0.137635,0.329765,0.152247,0.080846,0.322380,0.052199,0.164811,0.041286,0.165350,0.000000,0.002489,0.009077,0.000000,0.243336,0.254872,0.000000,0.005292,...,0.048666,0.022342,0.062368,0.145534,0.231190,0.026398,0.191689,0.350941,0.175405,0.334656,0.038711,0.000000,0.342110,0.074416,0.168863,0.185523,0.000000,0.258688,0.423096,0.159031,0.211030,0.063369,0.000000,0.289893,0.085181,0.233524,0.143641,0.068424,0.324288,0.373698,0.338794,0.080580,0.094924,0.079779,0.148607,0.071459,0.239955,0.139595,0.152497,0.313941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
user_user_df.idxmax(axis=1) #user-user maximum similarity pairs

0      915
1      700
2      862
3      749
4      306
      ... 
938    717
939    912
940    688
941    453
942    681
Length: 943, dtype: int64

#Getting results

####Run any item-similarity code from above to load recommendations. Results will be displayed for the last code selected. For this instance its cosine similarity.

In [ ]:
def movieIdToTitle(listMovieIDs):
    """
     Converting movieId to titles
    :param user_id: List of movies
    :return: movie titles
    """
    movie_titles= list()
    for id in listMovieIDs:
        movie_titles.append(movies[movies['movieid']==id]['title'])
    return movie_titles

In [ ]:
movieId_recommended=list()
def getRecommendedMoviesAsperUserSimilarity(userid):
   
    user2Movies= ratings[ratings['userid']== user_id]['movieid']
    sim_user=df_similar_user.iloc[0,0]
    df_recommended=pd.DataFrame(columns=['movieid','title','genres','userid','rating','timestamp'])
    for movieid in ratings[ratings['userid']== sim_user]['movieid']:
        if movieid not in user2Movies:
            df_new= ratings[(ratings.userid==sim_user) & (ratings.movieid==movieid)]
            df_recommended=pd.concat([df_recommended,df_new])
        best10=df_recommended.sort_values(['rating'], ascending = False )[1:10]  
    return best10['movieid']
  

In [ ]:
similar_user_series= ratings_user_matrix.idxmax(axis=1)
df_similar_user= similar_user_series.to_frame()

In [ ]:

df_similar_user.columns=['similarUser']

#Getting Recommendations for User ID=99

In [ ]:
#Getting recommendations for user
user_id=99
recommend_movies= movieIdToTitle(getRecommendedMoviesAsperUserSimilarity(user_id))
print("Movies you should watch are:\n")
print(recommend_movies)

Movies you should watch are:

[257    Contact (1997)
Name: title, dtype: object, 43    Dolores Claiborne (1994)
Name: title, dtype: object, 177    12 Angry Men (1957)
Name: title, dtype: object, 227    Star Trek: The Wrath of Khan (1982)
Name: title, dtype: object, 234    Mars Attacks! (1996)
Name: title, dtype: object, 201    Groundhog Day (1993)
Name: title, dtype: object, 47    Hoop Dreams (1994)
Name: title, dtype: object, 189    Henry V (1989)
Name: title, dtype: object, 215    When Harry Met Sally... (1989)
Name: title, dtype: object]


#------------------------------------------------------------------------

#ITEM-BASED COLLABORATIVE FILTERING

#ITEM-ITEM

In [ ]:
#USER-ITEM MATRIX
ratings_item_matrix=ratings.pivot_table(index=["movieid"],columns=["userid"],values="rating").reset_index(drop=True)
ratings_item_matrix.fillna(0,inplace=True)
ratings_item_matrix.head()

userid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
0,5.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,1.0,5.0,4.0,5.0,0.0,3.0,5.0,0.0,5.0,0.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,4.0,3.0,3.0,4.0,0.0,3.0,5.0,3.0,5.0,0.0,0.0,5.0,0.0,3.0,3.0,0.0,4.0,3.0,2.0,3.0,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
1,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,5.0,5.0,0.0,0.0,5.0,0.0,3.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ratings_item_matrix.fillna(0, inplace = True) #Replacing missing values with 0

In [ ]:
ratings_item_matrix.shape #943 users and 1682 movies

(1682, 943)

In [ ]:
user_similarity = 1 - pairwise_distances(ratings_user_matrix.to_numpy(), metric="jaccard" )
np.fill_diagonal(user_similarity, 0 ) 
user_user_df= pd.DataFrame(user_similarity)
user_user_df

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/pairwise.py:1738: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942
0,0.000000,0.056962,0.025157,0.024221,0.217984,0.248062,0.273585,0.114478,0.017301,0.203166,0.185864,0.117647,0.237057,0.149068,0.096210,0.215339,0.063830,0.279720,0.031802,0.095890,0.102689,0.186944,0.258929,0.122112,0.143791,0.114706,0.031250,0.135922,0.040816,0.067797,0.037037,0.075601,0.013699,0.010381,0.024138,0.010381,0.111486,0.135838,0.013793,0.033670,...,0.045902,0.040000,0.046823,0.129380,0.119741,0.027586,0.106667,0.138462,0.076159,0.231707,0.017241,0.010169,0.357143,0.044218,0.122754,0.204433,0.017065,0.143713,0.216463,0.067901,0.123810,0.034014,0.020979,0.149560,0.066667,0.099315,0.127946,0.067308,0.209906,0.291785,0.208672,0.050676,0.131148,0.064846,0.104651,0.049020,0.159021,0.035211,0.083333,0.232493
1,0.056962,0.000000,0.084112,0.088608,0.021552,0.132780,0.040268,0.052174,0.076923,0.069565,0.056522,0.056075,0.067278,0.118881,0.248120,0.063158,0.111111,0.059375,0.064935,0.037736,0.085586,0.027027,0.065000,0.101695,0.068702,0.224638,0.087500,0.052239,0.090909,0.129032,0.042553,0.119565,0.088608,0.123288,0.035714,0.064935,0.034783,0.033898,0.166667,0.141176,...,0.101010,0.108696,0.144444,0.142077,0.046154,0.086420,0.196078,0.012658,0.008772,0.089888,0.024096,0.157895,0.044077,0.154762,0.024845,0.167364,0.173333,0.102564,0.061798,0.133333,0.133858,0.021739,0.205882,0.064327,0.044444,0.057143,0.146789,0.308511,0.016779,0.033613,0.067873,0.160920,0.192982,0.243902,0.164384,0.110000,0.111842,0.090909,0.093023,0.040724
2,0.025157,0.084112,0.000000,0.200000,0.004386,0.039216,0.031603,0.066038,0.027027,0.034783,0.039823,0.029412,0.073095,0.027027,0.074830,0.031915,0.037975,0.012232,0.088235,0.030303,0.088785,0.016760,0.019900,0.051724,0.015385,0.087838,0.025974,0.055556,0.128205,0.054348,0.111111,0.079545,0.238095,0.104478,0.144928,0.088235,0.009091,0.029412,0.206349,0.253521,...,0.041237,0.146341,0.032609,0.086486,0.049180,0.052632,0.055556,0.006623,0.019048,0.056818,0.000000,0.194030,0.013661,0.022989,0.006410,0.124481,0.269841,0.044586,0.028409,0.103448,0.046154,0.131579,0.193548,0.023529,0.048780,0.019802,0.026316,0.105769,0.000000,0.012766,0.013333,0.021978,0.088889,0.068182,0.065789,0.019802,0.102740,0.055556,0.072581,0.009091
3,0.024221,0.088608,0.200000,0.000000,0.010152,0.026201,0.028916,0.106667,0.045455,0.019608,0.040609,0.041667,0.031250,0.033898,0.049180,0.031447,0.019608,0.010067,0.100000,0.074627,0.062827,0.034014,0.023392,0.095238,0.030303,0.056452,0.042553,0.061856,0.094340,0.063492,0.052632,0.065574,0.200000,0.100000,0.139535,0.047619,0.051948,0.028369,0.121951,0.134615,...,0.044118,0.084746,0.015625,0.049080,0.053763,0.041667,0.063291,0.016667,0.013158,0.054054,0.000000,0.111111,0.014881,0.035088,0.000000,0.061674,0.111111,0.038760,0.048611,0.042553,0.039216,0.076923,0.128205,0.043478,0.056604,0.028169,0.048193,0.062500,0.007605,0.019608,0.015385,0.016129,0.050633,0.103448,0.056000,0.013889,0.082645,0.095238,0.084211,0.021277
4,0.217984,0.021552,0.004386,0.010152,0.000000,0.122093,0.214286,0.109005,0.020725,0.108025,0.202703,0.071090,0.187408,0.100806,0.052830,0.145455,0.025253,0.170984,0.026316,0.115000,0.109718,0.267782,0.239544,0.075221,0.134529,0.072243,0.020408,0.159817,0.019512,0.063415,0.014423,0.058824,0.000000,0.005155,0.010101,0.000000,0.094340,0.218107,0.000000,0.009615,...,0.037383,0.004673,0.018868,0.076923,0.106667,0.005000,0.073059,0.176724,0.070423,0.167300,0.031250,0.000000,0.179856,0.029412,0.098814,0.091922,0.000000,0.135458,0.258333,0.068670,0.088983,0.024752,0.000000,0.184739,0.029851,0.087379,0.076923,0.026087,0.175141,0.264085,0.199313,0.028846,0.049669,0.023810,0.080153,0.032258,0.110236,0.031414,0.071730,0.207746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
item_similarity = 1 - pairwise_distances(ratings_item_matrix.to_numpy(), metric="l2" )
np.fill_diagonal(item_similarity, 0 ) 
item_item_df= pd.DataFrame(item_similarity)
item_item_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.000000,-76.723870,-79.074965,-75.896034,-80.565924,-83.734881,-70.007042,-76.006493,-77.625696,-81.752643,-76.967942,-81.534841,-77.128100,-82.588277,-70.721684,-81.643814,-81.000000,-83.882271,-84.076436,-82.180527,-78.768415,-76.562878,-83.587233,-74.332596,-70.049279,-82.312664,-82.222593,-69.213959,-78.037966,-83.622692,-75.694198,-81.969874,-80.926797,-83.427484,-83.041656,-83.279298,-83.575410,-77.892332,-80.847419,-81.207056,...,-83.628600,-83.782074,-83.776176,-83.852814,-83.758480,-83.752581,-83.758480,-83.776176,-83.776176,-83.793868,-83.923495,-83.723078,-83.705372,-83.770278,-83.723078,-83.687661,-83.723078,-83.681757,-83.746681,-83.876381,-83.799764,-83.593144,-83.799764,-83.799764,-83.829240,-83.829240,-83.799764,-83.829240,-83.782074,-83.728980,-83.723078,-83.870490,-83.829240,-83.799764,-83.723078,-83.782074,-83.829240,-83.799764,-83.687661,-83.687661
1,-76.723870,0.000000,-41.178193,-46.413078,-40.279535,-40.460825,-71.242647,-59.074953,-69.476947,-47.733972,-54.027266,-63.923031,-53.945427,-62.229740,-67.080834,-40.844952,-38.153544,-37.444766,-49.447993,-45.904158,-38.217343,-63.350602,-57.991525,-45.173586,-60.717096,-42.749286,-35.152455,-57.694122,-31.249031,-42.611925,-43.418465,-46.801674,-36.775654,-37.561639,-37.496753,-36.416574,-36.656341,-33.307434,-39.841156,-38.433488,...,-37.742741,-37.301436,-37.209946,-37.405729,-37.196859,-37.209946,-37.196859,-37.209946,-37.209946,-37.457769,-37.639358,-37.327536,-37.209946,-37.691084,-37.118237,-37.691084,-37.327536,-37.366652,-37.223030,-37.535698,-37.366652,-37.431758,-37.366652,-37.366652,-37.431758,-37.431758,-37.366652,-37.431758,-37.327536,-37.262253,-37.431758,-37.522721,-37.431758,-37.366652,-37.431758,-37.327536,-37.431758,-37.366652,-37.196859,-37.196859
2,-79.074965,-41.178193,0.000000,-51.124850,-38.408121,-33.727511,-71.097157,-61.577951,-67.117545,-43.609416,-56.870545,-68.526973,-48.899900,-57.043087,-64.726707,-33.380227,-33.351128,-30.256999,-42.520110,-38.357337,-35.945906,-69.823725,-56.956881,-44.617979,-58.816386,-37.509739,-33.263683,-64.513357,-36.563280,-36.000000,-47.620983,-41.497059,-35.496575,-29.561414,-30.701735,-30.480152,-30.384710,-39.620192,-36.802116,-34.355339,...,-30.685959,-30.048349,-30.224990,-30.368774,-30.112698,-30.032241,-30.112698,-30.224990,-30.224990,-30.112698,-30.368774,-29.983867,-29.838288,-30.432467,-30.112698,-30.432467,-29.919250,-29.903074,-29.886890,-30.240999,-30.032241,-30.622777,-30.032241,-30.032241,-30.112698,-30.112698,-30.032241,-30.112698,-29.983867,-30.096624,-30.112698,-30.224990,-30.112698,-30.032241,-30.016125,-29.983867,-30.112698,-30.032241,-30.112698,-29.822070
3,-75.896034,-46.413078,-51.124850,0.000000,-51.057660,-53.927225,-68.419018,-57.008620,-66.697858,-55.753854,-51.459508,-59.074953,-54.919585,-64.726707,-69.306472,-52.768020,-48.729267,-52.084838,-58.304300,-54.623736,-52.122500,-60.212744,-56.913729,-53.662601,-61.056426,-49.239427,-48.668904,-58.203040,-47.620983,-52.516353,-50.058790,-51.507142,-47.218254,-51.754147,-52.310412,-51.402290,-51.630789,-47.425200,-46.916594,-49.507425,...,-52.581713,-52.188345,-52.178943,-52.301032,-52.150729,-52.141321,-52.150729,-52.178943,-52.178943,-52.282267,-52.413481,-52.188345,-52.103672,-52.075418,-52.263496,-52.450912,-52.113087,-52.065997,-52.188345,-52.338541,-52.216539,-52.188345,-52.216539,-52.216539,-52.263496,-52.263496,-52.216539,-52.263496,-52.188345,-52.141321,-52.263496,-52.329167,-51.981129,-52.028294,-52.150729,-52.188345,-52.263496,-52.216539,-52.094256,-52.037722
4,-80.565924,-40.279535,-38.408121,-51.057660,0.000000,-35.646964,-72.423430,-60.016391,-67.315445,-46.738873,-56.515215,-66.786429,-51.469038,-59.92

In [ ]:
item_similarity = 1 - pairwise_distances(ratings_item_matrix.to_numpy(), metric="correlation" )
np.fill_diagonal(item_similarity, 0 ) 
item_item_df= pd.DataFrame(item_similarity)
item_item_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.000000,0.234595,0.193362,0.226213,0.128840,0.015113,0.347354,0.254490,0.209502,0.104655,0.221811,0.171780,0.198063,0.089739,0.339027,0.129048,0.108860,-0.023226,0.022113,0.080876,0.212938,0.256862,0.072484,0.281745,0.333410,0.064470,0.072615,0.376461,0.204595,0.028232,0.257416,0.097403,0.116531,0.051778,0.100337,0.047863,0.017424,0.202305,0.117591,0.127061,...,0.032456,-0.025547,0.002251,-0.022401,0.002251,0.002251,0.002251,0.002251,0.002251,-0.025502,-0.029676,0.050142,0.034179,0.005893,0.018215,0.015635,0.050142,0.050142,0.018215,-0.035996,-0.029676,0.036156,-0.029676,-0.029676,-0.029676,-0.029676,-0.029676,-0.029676,-0.029676,0.003186,0.018215,-0.029676,-0.029676,-0.029676,0.018215,-0.029676,-0.029676,-0.029676,0.034179,0.034179
1,0.234595,0.000000,0.190649,0.409044,0.240712,0.030062,0.220022,0.206020,0.077894,0.072906,0.362190,0.343212,0.074405,-0.032029,0.076977,0.031636,0.320204,0.085478,-0.069434,-0.001471,0.244859,0.368449,0.147752,0.338009,0.146921,0.122634,0.358680,0.371013,0.526570,0.022024,0.375543,0.066868,0.415814,0.047800,0.058064,0.189913,0.165874,0.503101,0.258495,0.192115,...,0.029341,0.020810,0.071415,0.032905,0.071415,0.071415,0.071415,0.071415,0.071415,-0.018783,-0.012451,-0.012451,0.071415,-0.016191,0.099371,-0.020829,-0.012451,-0.012451,0.099371,-0.015103,-0.012451,0.056823,-0.012451,-0.012451,-0.012451,-0.012451,-0.012451,-0.012451,-0.012451,0.041716,-0.012451,-0.012451,-0.012451,-0.012451,-0.012451,-0.012451,-0.012451,-0.012451,0.071415,0.071415
2,0.193362,0.190649,0.000000,0.227849,0.141368,0.065347,0.258855,0.078636,0.146181,0.079608,0.265441,0.207190,0.172782,0.078041,0.138165,0.119143,0.349635,0.104404,0.032821,0.122217,0.124356,0.170528,0.127550,0.301930,0.189111,0.148284,0.208946,0.144225,0.176465,0.075252,0.166181,0.113296,0.347554,0.164012,0.020445,0.044157,0.028269,0.159152,0.211717,0.118822,...,0.019819,-0.013106,-0.009764,-0.013677,-0.009764,-0.009764,-0.009764,-0.009764,-0.009764,-0.004554,-0.009764,-0.009764,0.091421,-0.012697,-0.009764,-0.016334,0.057693,0.057693,0.091421,-0.011844,-0.009764,0.001141,-0.009764,-0.009764,-0.009764,-0.009764,-0.009764,-0.009764,-0.009764,-0.013816,-0.009764,-0.009764,-0.009764,-0.009764,0.023964,-0.009764,-0.009764,-0.009764,-0.009764,0.091421
3,0.226213,0.409044,0.227849,0.000000,0.237298,0.021878,0.295489,0.352800,0.229922,0.138220,0.473171,0.457358,0.254039,0.090185,0.113703,0.092877,0.319291,0.065539,0.012338,0.104686,0.151499,0.434904,0.312218,0.278431,0.238576,0.300564,0.305548,0.384661,0.335032,0.147436,0.368948,0.261366,0.386864,0.104171,0.023100,0.136980,0.118756,0.362505,0.381267,0.261676,...,-0.004804,-0.003407,0.025622,0.002080,0.025622,0.025622,0.025622,0.025622,0.025622,-0.025071,-0.016619,-0.016619,0.046743,0.056858,-0.016619,-0.027801,0.067863,0.067863,-0.016619,-0.020158,-0.016619,0.041604,-0.016619,-0.016619,-0.016619,-0.016619,-0.016619,-0.016619,-0.016619,0.021312,-0.016619,-0.016619,0.088984,0.088984,0.025622,-0.016619,-0.016619,-0.016619,0.046743,0.067863
4,0.128840,0.240712,0.141368,0.237298,0.000000,-0.008594,0.205289,0.145866,0.142541,-0.033746,0.281260,0.272113,0.084938,-0.024039,0.134558,0.059505,0.332488,0.052929,0.013776,-0.015551,0.121435,0.200503,0.139066,0.134909,0.159652,0.163444,0.188386,0.213164,0.250817,0.076392,0.255294,0.075909,0.210477,0.092394,0.014861,0.191649,0.036030,0.326225,0.221898,0.221035,...,0.022712,0.016108,0.055782,0.025571,0.055782,0.055782,0.055782,0.055782,0.055782,-0.014918,-0.009889,-0.009889,0.088618,-0.012859,0.088618,-0.016543,-0.009889,-0.009889,-0.009889,-0.011995,-0.009889,0.037974,-0.009889,-0.009889,-0.009889,-0.009889,-0.009889,-0.009889,-0.009889,-0.013993,-0.009889,-0.009889,-0

In [ ]:
item_similarity = 1 - pairwise_distances(ratings_item_matrix.to_numpy(), metric="euclidean" )
np.fill_diagonal(item_similarity, 0 ) 
item_item_df= pd.DataFrame(item_similarity)
item_item_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.000000,-76.723870,-79.074965,-75.896034,-80.565924,-83.734881,-70.007042,-76.006493,-77.625696,-81.752643,-76.967942,-81.534841,-77.128100,-82.588277,-70.721684,-81.643814,-81.000000,-83.882271,-84.076436,-82.180527,-78.768415,-76.562878,-83.587233,-74.332596,-70.049279,-82.312664,-82.222593,-69.213959,-78.037966,-83.622692,-75.694198,-81.969874,-80.926797,-83.427484,-83.041656,-83.279298,-83.575410,-77.892332,-80.847419,-81.207056,...,-83.628600,-83.782074,-83.776176,-83.852814,-83.758480,-83.752581,-83.758480,-83.776176,-83.776176,-83.793868,-83.923495,-83.723078,-83.705372,-83.770278,-83.723078,-83.687661,-83.723078,-83.681757,-83.746681,-83.876381,-83.799764,-83.593144,-83.799764,-83.799764,-83.829240,-83.829240,-83.799764,-83.829240,-83.782074,-83.728980,-83.723078,-83.870490,-83.829240,-83.799764,-83.723078,-83.782074,-83.829240,-83.799764,-83.687661,-83.687661
1,-76.723870,0.000000,-41.178193,-46.413078,-40.279535,-40.460825,-71.242647,-59.074953,-69.476947,-47.733972,-54.027266,-63.923031,-53.945427,-62.229740,-67.080834,-40.844952,-38.153544,-37.444766,-49.447993,-45.904158,-38.217343,-63.350602,-57.991525,-45.173586,-60.717096,-42.749286,-35.152455,-57.694122,-31.249031,-42.611925,-43.418465,-46.801674,-36.775654,-37.561639,-37.496753,-36.416574,-36.656341,-33.307434,-39.841156,-38.433488,...,-37.742741,-37.301436,-37.209946,-37.405729,-37.196859,-37.209946,-37.196859,-37.209946,-37.209946,-37.457769,-37.639358,-37.327536,-37.209946,-37.691084,-37.118237,-37.691084,-37.327536,-37.366652,-37.223030,-37.535698,-37.366652,-37.431758,-37.366652,-37.366652,-37.431758,-37.431758,-37.366652,-37.431758,-37.327536,-37.262253,-37.431758,-37.522721,-37.431758,-37.366652,-37.431758,-37.327536,-37.431758,-37.366652,-37.196859,-37.196859
2,-79.074965,-41.178193,0.000000,-51.124850,-38.408121,-33.727511,-71.097157,-61.577951,-67.117545,-43.609416,-56.870545,-68.526973,-48.899900,-57.043087,-64.726707,-33.380227,-33.351128,-30.256999,-42.520110,-38.357337,-35.945906,-69.823725,-56.956881,-44.617979,-58.816386,-37.509739,-33.263683,-64.513357,-36.563280,-36.000000,-47.620983,-41.497059,-35.496575,-29.561414,-30.701735,-30.480152,-30.384710,-39.620192,-36.802116,-34.355339,...,-30.685959,-30.048349,-30.224990,-30.368774,-30.112698,-30.032241,-30.112698,-30.224990,-30.224990,-30.112698,-30.368774,-29.983867,-29.838288,-30.432467,-30.112698,-30.432467,-29.919250,-29.903074,-29.886890,-30.240999,-30.032241,-30.622777,-30.032241,-30.032241,-30.112698,-30.112698,-30.032241,-30.112698,-29.983867,-30.096624,-30.112698,-30.224990,-30.112698,-30.032241,-30.016125,-29.983867,-30.112698,-30.032241,-30.112698,-29.822070
3,-75.896034,-46.413078,-51.124850,0.000000,-51.057660,-53.927225,-68.419018,-57.008620,-66.697858,-55.753854,-51.459508,-59.074953,-54.919585,-64.726707,-69.306472,-52.768020,-48.729267,-52.084838,-58.304300,-54.623736,-52.122500,-60.212744,-56.913729,-53.662601,-61.056426,-49.239427,-48.668904,-58.203040,-47.620983,-52.516353,-50.058790,-51.507142,-47.218254,-51.754147,-52.310412,-51.402290,-51.630789,-47.425200,-46.916594,-49.507425,...,-52.581713,-52.188345,-52.178943,-52.301032,-52.150729,-52.141321,-52.150729,-52.178943,-52.178943,-52.282267,-52.413481,-52.188345,-52.103672,-52.075418,-52.263496,-52.450912,-52.113087,-52.065997,-52.188345,-52.338541,-52.216539,-52.188345,-52.216539,-52.216539,-52.263496,-52.263496,-52.216539,-52.263496,-52.188345,-52.141321,-52.263496,-52.329167,-51.981129,-52.028294,-52.150729,-52.188345,-52.263496,-52.216539,-52.094256,-52.037722
4,-80.565924,-40.279535,-38.408121,-51.057660,0.000000,-35.646964,-72.423430,-60.016391,-67.315445,-46.738873,-56.515215,-66.786429,-51.469038,-59.92

In [ ]:
item_similarity = 1 - pairwise_distances(ratings_item_matrix.to_numpy(), metric="l1" )
np.fill_diagonal(item_similarity, 0 ) 
item_item_df= pd.DataFrame(item_similarity)
item_item_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.0,-1546.0,-1605.0,-1538.0,-1666.0,-1755.0,-1367.0,-1521.0,-1589.0,-1697.0,-1558.0,-1701.0,-1563.0,-1746.0,-1373.0,-1683.0,-1675.0,-1760.0,-1773.0,-1710.0,-1612.0,-1553.0,-1774.0,-1464.0,-1365.0,-1728.0,-1717.0,-1329.0,-1608.0,-1752.0,-1519.0,-1695.0,-1681.0,-1743.0,-1734.0,-1740.0,-1744.0,-1599.0,-1668.0,-1681.0,...,-1747.0,-1753.0,-1752.0,-1755.0,-1751.0,-1750.0,-1751.0,-1752.0,-1752.0,-1755.0,-1757.0,-1751.0,-1750.0,-1753.0,-1749.0,-1749.0,-1751.0,-1750.0,-1751.0,-1757.0,-1754.0,-1747.0,-1754.0,-1754.0,-1755.0,-1755.0,-1754.0,-1755.0,-1753.0,-1752.0,-1749.0,-1756.0,-1755.0,-1754.0,-1749.0,-1753.0,-1755.0,-1754.0,-1749.0,-1749.0
1,-1546.0,0.0,-522.0,-641.0,-493.0,-480.0,-1354.0,-918.0,-1264.0,-636.0,-809.0,-1032.0,-836.0,-1011.0,-1208.0,-498.0,-450.0,-425.0,-670.0,-609.0,-477.0,-1034.0,-885.0,-617.0,-1054.0,-543.0,-388.0,-912.0,-353.0,-523.0,-560.0,-614.0,-422.0,-426.0,-427.0,-411.0,-405.0,-378.0,-481.0,-452.0,...,-428.0,-420.0,-417.0,-420.0,-416.0,-417.0,-416.0,-417.0,-417.0,-424.0,-424.0,-420.0,-417.0,-426.0,-416.0,-428.0,-420.0,-421.0,-418.0,-424.0,-421.0,-422.0,-421.0,-421.0,-422.0,-422.0,-421.0,-422.0,-420.0,-419.0,-422.0,-423.0,-422.0,-421.0,-422.0,-420.0,-422.0,-421.0,-416.0,-416.0
2,-1605.0,-522.0,0.0,-750.0,-442.0,-331.0,-1341.0,-963.0,-1159.0,-513.0,-874.0,-1135.0,-693.0,-840.0,-1103.0,-335.0,-355.0,-278.0,-491.0,-428.0,-404.0,-1203.0,-836.0,-596.0,-979.0,-420.0,-349.0,-1085.0,-448.0,-372.0,-651.0,-485.0,-393.0,-271.0,-290.0,-288.0,-284.0,-495.0,-414.0,-365.0,...,-281.0,-275.0,-276.0,-279.0,-275.0,-274.0,-275.0,-276.0,-276.0,-275.0,-277.0,-273.0,-270.0,-279.0,-275.0,-281.0,-271.0,-270.0,-271.0,-277.0,-274.0,-283.0,-274.0,-274.0,-275.0,-275.0,-274.0,-275.0,-273.0,-276.0,-275.0,-276.0,-275.0,-274.0,-273.0,-273.0,-275.0,-274.0,-275.0,-269.0
3,-1538.0,-641.0,-750.0,0.0,-745.0,-788.0,-1294.0,-892.0,-1210.0,-842.0,-759.0,-932.0,-874.0,-1093.0,-1288.0,-776.0,-686.0,-743.0,-896.0,-815.0,-783.0,-988.0,-881.0,-835.0,-1068.0,-695.0,-672.0,-952.0,-681.0,-755.0,-730.0,-744.0,-648.0,-734.0,-747.0,-731.0,-733.0,-666.0,-645.0,-698.0,...,-752.0,-742.0,-741.0,-744.0,-740.0,-739.0,-740.0,-741.0,-741.0,-746.0,-746.0,-742.0,-739.0,-740.0,-744.0,-750.0,-740.0,-739.0,-742.0,-746.0,-743.0,-742.0,-743.0,-743.0,-744.0,-744.0,-743.0,-744.0,-742.0,-741.0,-744.0,-745.0,-738.0,-739.0,-740.0,-742.0,-744.0,-743.0,-738.0,-738.0
4,-1666.0,-493.0,-442.0,-745.0,0.0,-366.0,-1378.0,-916.0,-1170.0,-586.0,-839.0,-1074.0,-760.0,-921.0,-1118.0,-372.0,-366.0,-297.0,-510.0,-497.0,-423.0,-1174.0,-825.0,-699.0,-1016.0,-413.0,-352.0,-1028.0,-413.0,-377.0,-588.0,-510.0,-456.0,-286.0,-301.0,-279.0,-291.0,-414.0,-409.0,-338.0,...,-294.0,-284.0,-283.0,-286.0,-282.0,-281.0,-282.0,-283.0,-283.0,-288.0,-288.0,-284.0,-281.0,-290.0,-280.0,-292.0,-284.0,-285.0,-284.0,-288.0,-285.0,-290.0,-285.0,-285.0,-286.0,-286.0,-285.0,-286.0,-284.0,-287.0,-286.0,-287.0,-286.0,-285.0,-286.0,-284.0,-286.0,-285.0,-286.0,-280.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,-1753.0,-420.0,-273.0,-742.0,-284.0,-93.0,-1489.0,-875.0,-1165.0,-341.0,-908.0,-1171.0,-629.0,-726.0,-1107.0,-125.0,-287.0,-28.0,-273.0,-246.0,-232.0,-1233.0,-750.0,-600.0,-1009.0,-252.0,-177.0,-1085.0,-304.0,-146.0,-559.0,-307.0,-335.0,-19.0,-24.0,-28.0,-18.0,-361.0,-284.0,-165.0,...,-15.0,-3.0,-4.0,-7.0,-3.0,-2.0,-3.0,-4.0,-4.0,-5.0,-5.0,-1.0,-2.0,-7.0,-3.0,-9.0,-1.0,-2.0,-1.0,-5.0,-2.0,-13.0,-2.0,-2.0,-3.0,-3.0,-2.0,-3.0,-

In [ ]:
item_similarity = 1 - pairwise_distances(ratings_item_matrix.to_numpy(), metric="cosine" )
np.fill_diagonal(item_similarity, 0 ) 
item_item_df= pd.DataFrame(item_similarity)
item_item_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.000000,0.402382,0.330245,0.454938,0.286714,0.116344,0.620979,0.481114,0.496288,0.273935,0.468291,0.460392,0.417509,0.347678,0.574377,0.224120,0.273916,0.046232,0.191772,0.232930,0.339196,0.527169,0.338105,0.467598,0.567950,0.227295,0.208472,0.592877,0.361657,0.150959,0.439441,0.259257,0.288536,0.090654,0.136757,0.108560,0.069805,0.368420,0.278386,0.246123,...,0.065390,0.010550,0.023592,0.014155,0.023592,0.023592,0.023592,0.023592,0.023592,0.014226,0.0,0.058979,0.047183,0.032856,0.035387,0.048189,0.058979,0.058979,0.035387,0.0,0.0,0.067404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033363,0.035387,0.0,0.000000,0.000000,0.035387,0.0,0.0,0.0,0.047183,0.047183
1,0.402382,0.000000,0.273069,0.502571,0.318836,0.083563,0.383403,0.337002,0.255252,0.171082,0.468506,0.459946,0.213972,0.125463,0.253162,0.098185,0.390438,0.113063,0.030692,0.091550,0.319308,0.483349,0.278182,0.434907,0.306639,0.206481,0.408737,0.483077,0.581119,0.089355,0.462797,0.160738,0.479187,0.072342,0.087347,0.215247,0.184592,0.562970,0.334570,0.256668,...,0.049324,0.035017,0.078299,0.046980,0.078299,0.078299,0.078299,0.078299,0.078299,0.000000,0.0,0.000000,0.078299,0.000000,0.104399,0.000000,0.000000,0.000000,0.104399,0.0,0.0,0.074571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055366,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.078299,0.078299
2,0.330245,0.273069,0.000000,0.324866,0.212957,0.106722,0.372921,0.200794,0.273669,0.158104,0.361165,0.319295,0.271402,0.190242,0.266335,0.167609,0.404239,0.126564,0.105377,0.187733,0.194546,0.293732,0.233814,0.381910,0.306447,0.214125,0.261303,0.269296,0.253074,0.126347,0.259129,0.185187,0.404930,0.179008,0.046315,0.073332,0.051270,0.240534,0.277311,0.176729,...,0.036615,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009736,0.0,0.000000,0.096875,0.000000,0.000000,0.000000,0.064583,0.064583,0.096875,0.0,0.0,0.018452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.032292,0.0,0.0,0.0,0.000000,0.096875
3,0.454938,0.502571,0.324866,0.000000,0.334239,0.090308,0.489283,0.490236,0.419044,0.252561,0.588337,0.584884,0.397251,0.266764,0.330457,0.169240,0.404889,0.100856,0.129363,0.208262,0.256467,0.572811,0.446568,0.413422,0.422300,0.380248,0.370192,0.531380,0.429785,0.216894,0.480685,0.351676,0.467377,0.127933,0.062932,0.170815,0.143852,0.457528,0.456760,0.331256,...,0.023691,0.016819,0.037609,0.022565,0.037609,0.037609,0.037609,0.037609,0.037609,0.000000,0.0,0.000000,0.056413,0.069838,0.000000,0.000000,0.075218,0.075218,0.000000,0.0,0.0,0.064472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.039890,0.000000,0.0,0.094022,0.094022,0.037609,0.0,0.0,0.0,0.056413,0.075218
4,0.286714,0.318836,0.212957,0.334239,0.000000,0.037299,0.334769,0.259161,0.272448,0.055453,0.375809,0.373824,0.196495,0.103288,0.265123,0.112156,0.389191,0.077737,0.088631,0.061197,0.192679,0.319438,0.245088,0.237436,0.284242,0.228855,0.242739,0.326892,0.321288,0.127992,0.338714,0.151706,0.280778,0.110782,0.041288,0.213947,0.058926,0.391974,0.287402,0.272573,...,0.039565,0.028088,0.062807,0.037684,0.062807,0.062807,0.062807,0.062807,0.062807,0.000000,0.0,0.000000,0.094211,0.000000,0.094211,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.053835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.094211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [ ]:
item_item_df.idxmax(axis=1) #item-item maximum similarity pairs

0         49
1        232
2        409
3         55
4        217
        ... 
1677    1678
1678    1677
1679    1677
1680    1671
1681     766
Length: 1682, dtype: int64

##Getting Results- 

####Run any item-similarity code from above to load recommendations. Results will be displayed for the last code selected. For this instance its cosine similarity.

In [ ]:
movie_name="Fargo (1996)"
movieid=movies[movies["title"]==movie_name].index
movieid=movieid[0]

In [ ]:
movies["Similarity"]=item_similarity[movieid]

#Getting Recommendations for Movie "Fargo" from 1996

In [ ]:
print("Movies you should watch are:\n")
movies.sort_values("Similarity",ascending=False)[1:10]

Movies you should watch are:



,movieid,title,Similarity
49,50,Star Wars (1977),0.686533
126,127,"Godfather, The (1972)",0.646271
55,56,Pulp Fiction (1994),0.640382
97,98,"Silence of the Lambs, The (1991)",0.635430
0,1,Toy Story (1995),0.630601
180,181,Return of the Jedi (1983),0.629361
236,237,Jerry Maguire (1996),0.624375
275,276,Leaving Las Vegas (1995),0.621995
8,9,Dead Man Walking (1995),0.618001


#---------------------------------------------------------------------

#EVALUATION

#RMSD for CF

In [ ]:
ratings #how many values we have - nije vazno

,userid,movieid,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)
...,...,...,...,...,...
99995,840,1674,4,891211682,Mamma Roma (1962)
99996,655,1640,3,888474646,"Eighth Day, The (1996)"
99997,655,1637,3,888984255,Girls Town (1996)
99998,655,1630,3,887428735,"Silence of the Palace, The (Saimt el Qusur) (1..."


In [ ]:
#small fraction of dateset
ratingsmall=ratings.sample(frac=0.1, random_state=42)
ratings_new= ratingsmall.loc[:, ['userid', 'movieid','rating']]

In [ ]:
train_data = ratings_new.sample(frac=0.8,random_state=200)
test_data=ratings_new.drop(train_data.index)
test_data.size

6000

In [ ]:
train_data_matrix=train_data.pivot_table(index=["movieid"],columns=["userid"],values="rating").reset_index(drop=True)
train_data_matrix.fillna(0,inplace=True)
train_data_matrix.head()

userid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,30,31,32,34,35,37,38,39,40,41,42,43,...,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,942,943
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_data_matrix=test_data.pivot_table(index=["movieid"],columns=["userid"],values="rating").reset_index(drop=True)
test_data_matrix.fillna(0,inplace=True)
test_data_matrix.head()

userid,1,4,5,6,7,8,10,11,12,13,14,15,16,17,18,20,21,22,23,24,25,26,28,30,31,32,35,37,38,40,41,42,43,44,45,47,48,49,51,52,...,885,886,887,889,890,892,894,895,896,897,898,899,900,901,902,903,905,906,907,910,911,912,914,916,917,918,919,921,922,923,924,927,928,932,933,934,936,938,939,940
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


####Rename metric to other similarities to get results.

In [ ]:
#Rename here
# User Similarity Matrix
user_correlation = 1 - pairwise_distances(test_data_matrix, metric='cosine')  #<---rename to euclidean
user_correlation[np.isnan(user_correlation)] = 0
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(test_data_matrix.T, metric='cosine') #<---rename to euclidean
item_correlation[np.isnan(item_correlation)] = 0

In [ ]:
# Function to predict ratings
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [ ]:
test_data_matrix.shape

(777, 652)

In [ ]:
 #Remove hash or put hash if code doesnt work--->
 test_data_matrix=test_data_matrix.to_numpy()

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSD
def rmsd(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))
# Predict ratings on the training data with both similarity score
user_prediction = predict(test_data_matrix, user_correlation, type='user')
item_prediction = predict(test_data_matrix, item_correlation, type='item')
# RMSD on the train data
print('User-based CF RMSD: ' + str(rmsd(user_prediction, test_data_matrix)))
print('Item-based CF RMSD: ' + str(rmsd(item_prediction, test_data_matrix)))



User-based CF RMSD: 2.1871728795229455
Item-based CF RMSD: 2.09594988703466


##RMSD Values are still quite poor. So, building a better model and then we will evaluate RMSD & Recall.

#-----------------------------------------------------------------------

#MovienChill!- Our Recommender System

#Feature Selection/ Extraction

##Deriving features from movie, genre & user tables

In [ ]:
movies

,movieid,title,Similarity
0,1,Toy Story (1995),0.630601
1,2,GoldenEye (1995),0.350519
2,3,Four Rooms (1995),0.349424
3,4,Get Shorty (1995),0.505085
4,5,Copycat (1995),0.306193
...,...,...,...
1677,1678,Mat' i syn (1997),0.000000
1678,1679,B. Monkey (1998),0.000000
1679,1680,Sliding Doors (1998),0.000000
1680,1681,You So Crazy (1994),0.031184


In [ ]:
o_cols=['occupation','id']
occup = pd.read_csv('ml-100k/u.occupation', sep='|', names=o_cols,
                  )
occup['id'] = range(0, 0+len(occup))

occup

,occupation,id
0,administrator,0
1,artist,1
2,doctor,2
3,educator,3
4,engineer,4
5,entertainment,5
6,executive,6
7,healthcare,7
8,homemaker,8
9,lawyer,9


In [ ]:
g_cols=['genre', 'movieid']
genre = pd.read_csv('ml-100k/u.genre', sep='|', names=g_cols,
                  )

genre

,genre,movieid
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4
5,Comedy,5
6,Crime,6
7,Documentary,7
8,Drama,8
9,Fantasy,9


In [ ]:
# pass in column names for each CSV
u_cols = ['userid', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,
                  )
lens = pd.merge(users, ratings)
lens = pd.merge(lens,genre)
lens=pd.merge(lens, occup)

lens

,userid,age,sex,occupation,zip_code,movieid,rating,timestamp,title,genre,id
0,1,24,M,technician,85711,1,5,874965758,Toy Story (1995),Action,19
1,44,26,M,technician,46260,1,4,878341315,Toy Story (1995),Action,19
2,77,30,M,technician,29379,1,5,884732808,Toy Story (1995),Action,19
3,244,28,M,technician,80525,1,4,880604405,Toy Story (1995),Action,19
4,294,34,M,technician,92110,1,5,877819634,Toy Story (1995),Action,19
...,...,...,...,...,...,...,...,...,...,...,...
3292,693,43,F,healthcare,85210,9,3,875481856,Dead Man Walking (1995),Fantasy,7
3293,735,29,F,healthcare,85719,9,4,876698755,Dead Man Walking (1995),Fantasy,7
3294,910,28,M,healthcare,29301,9,4,880821079,Dead Man Walking (1995),Fantasy,7
3295,655,50,F,healthcare,60657,6,4,887425812,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime,7


In [ ]:
lens['genre'] = pd.factorize(lens.genre)[0] + 1
lens

lens['sex'] = pd.factorize(lens.sex)[0] + 1
lens

lens['occupation'] = pd.factorize(lens.occupation)[0] + 1
lens

,userid,age,sex,occupation,zip_code,movieid,rating,timestamp,title,genre,id
0,1,24,1,1,85711,1,5,874965758,Toy Story (1995),1,19
1,44,26,1,1,46260,1,4,878341315,Toy Story (1995),1,19
2,77,30,1,1,29379,1,5,884732808,Toy Story (1995),1,19
3,244,28,1,1,80525,1,4,880604405,Toy Story (1995),1,19
4,294,34,1,1,92110,1,5,877819634,Toy Story (1995),1,19
...,...,...,...,...,...,...,...,...,...,...,...
3292,693,43,2,21,85210,9,3,875481856,Dead Man Walking (1995),16,7
3293,735,29,2,21,85719,9,4,876698755,Dead Man Walking (1995),16,7
3294,910,28,1,21,29301,9,4,880821079,Dead Man Walking (1995),16,7
3295,655,50,2,21,60657,6,4,887425812,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,17,7


#The final input table for training

In [ ]:
f = lens.iloc[:,[0,1,2,3,5,9]]
l= lens.iloc[:,[6]]
f.tail(10)

,userid,age,sex,occupation,movieid,genre
3287,60,50,1,21,9,16
3288,164,47,1,21,9,16
3289,401,46,2,21,9,16
3290,630,26,2,21,9,16
3291,655,50,2,21,9,16
3292,693,43,2,21,9,16
3293,735,29,2,21,9,16
3294,910,28,1,21,9,16
3295,655,50,2,21,6,17
3296,655,50,2,21,18,18


#Therefore Features selected: userid, age, sex, occupation, movieid, genre

#The Output table for training containing only "rating"

In [ ]:
l.tail(10)

,rating
3287,5
3288,4
3289,3
3290,2
3291,3
3292,3
3293,4
3294,4
3295,4
3296,3


In [ ]:
features=f.values
label=l.values.ravel()
label

array([5, 4, 5, ..., 4, 4, 3])

In [ ]:
label.shape

(3297,)

#Train & test using Random Forest gave best **performance**

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(features,label,test_size=0.2, random_state=232 )

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import metrics
features,label = make_classification(n_samples=len(features), n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)

model = RandomForestClassifier(max_depth=2, random_state=0)
model.fit(X_train,Y_train)



RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
y_pred=model.predict(X_test)
training_score = model.score(X_train,Y_train)
testing_score= model.score(X_test,Y_test)

#Evaluation

In [ ]:
print('Training score:',training_score)
print('Testing score:',testing_score)

print("Accuracy:",metrics.accuracy_score(Y_test, y_pred))


Training score: 0.3856655290102389
Testing score: 0.38484848484848483
Accuracy: 0.38484848484848483


#Compute the RMSD & RECALL

The RMSD of predicted values for times t of a regression's dependent variable is computed
for n different predictions as the square root of the mean of the squares of the deviations:

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
cm

array([[  0,   0,   0,  22,   0],
       [  0,   0,   0,  64,   0],
       [  0,   0,   0, 152,   1],
       [  0,   0,   0, 252,   3],
       [  0,   0,   0, 164,   2]])

In [ ]:
print('RMSD: ' + str(rmsd(y_pred, Y_test)))


RMSD: 1.0850219939119794


In [ ]:
from sklearn.metrics import recall_score
recall_score(Y_test, y_pred, average=None)

array([0.        , 0.        , 0.        , 0.98823529, 0.01204819])

In [ ]:
# calculate recall
recall = recall_score(Y_test, y_pred, average='micro')
print('Recall: %.3f' % recall)

Recall: 0.385


#Great accuracy values!
##End of Code

#----------------------------------------------------------------